In [1]:
import numpy as np
import sys
from tqdm import tqdm

sys.path.append('../src')
from models import Vicsek
from util import grab_plot, plot, VideoWriter

In [3]:
N_ANTS = 100
PROXIMITY = 20.
VELOCITY = 3.
UPDATE_RATE = 1.
VARIANCE = .3
WINDOW_SIZE = 500
ANT_SIZE = 50.
ANT_COLOR = 'red'
ANT_OPACITY = .15
FARM_COLOR = '#1b1f22'
N_STEPS = 5000
FILENAME = '_autoplay.mp4'
FRAME_RATE = 30.
FADE = 25

x = np.random.rand(N_ANTS, 2)*WINDOW_SIZE
v = 2*np.pi*np.random.rand(N_ANTS, 1)
model = Vicsek(PROXIMITY, VELOCITY, UPDATE_RATE, VARIANCE, WINDOW_SIZE)

with VideoWriter(FILENAME, WINDOW_SIZE, FRAME_RATE) as vid:
    for i in tqdm(range(N_STEPS), desc='Ant Farm'):
        x, v = model(x, v)
        opacity = ANT_OPACITY*(i/FADE) if i < FADE else ANT_OPACITY*((N_STEPS - i)/FADE) if i > N_STEPS - FADE else ANT_OPACITY
        plot(x, size=ANT_SIZE, color=ANT_COLOR, opacity=opacity, winsize=WINDOW_SIZE, facecolor=FARM_COLOR)
        vid.write(grab_plot())

Ant Farm: 100%|██████████| 5000/5000 [01:24<00:00, 59.46it/s]


<Figure size 640x480 with 0 Axes>